# Groundwater notebook

The Groundwater model describes the dynamics of the subsurface water at the county-scale. The two main phenomenon are the evolution of piezometric head (function of recharge, withdrawals and lateral flows) and the lateral flows (exchange of flow due to the difference in head between two adjacent counties).

# The aquifer component

(problem with my julia kernel - in plain text for now)

@defcomp Aquifer begin
  aquifers = Index()

  #Aquifer description
  depthaquif = Parameter(index=[aquifers], units="1 m")
  areaaquif = Parameter(index=[aquifers], units="1000 m^3")
  storagecoef = Parameter(index=[aquifers], units="")

  #Recharge
  recharge = Parameter(index=[aquifers, time], units="1000m^3")

  #Withdrawals - to be optimised
  withdrawal = Parameter(index=[aquifers, time], units="1000m^3")

  #Lateral flows
  lateralflows = Variable(index=[aquifers, time], units="1000m^3)
  aquiferconnexion = Parameter(index=[aquifers, aquifers], units ="") # aquiferconnexion[aa,aa']=1 -> aquifers are connected, 0 otherwise.
  lateralconductivity = Parameter(index=[aquifers, aquifers], units="1000m^2/month") ## how should I specify per month per year?
  deltatime = Parameter("units=month")

  #Piezometric head
  piezohead0 = Parameter(index=[aquifers], units="1 m") # used for initialisation
  piezohead = Variable(index=[aquifers, time], units="1 m")
end

# Model equations

The two equations at the heart of the component are the one describing the evolution of the piezometric head (measured as the distance between the county elevation and the water table - drawdown would be more appropriate):

piezohead[a,t] = piezohead[a,t-1] + 1/(storagecoef[a]*areaaquif[a])*(- recharge[a,t] + withdrawal[a,t] + lateralflows[a,t])

And the one describing the lateral flows:

lateralflows[a,t] = \sum_{a\*} lateralconductivity[a,a\*] \* (piezohead[a\*,tt-1]-piezohead[a,tt-1])

# Functions

Groundwater.jl contains a serie of functions:
    - The timestep function computes the variables evolution using the two precedently described equations
    - The constraints functions min and max ensure that the piezohead > 0 (non-artesian well) and piezohead < depthaquifer (i.e. confined aquifers remain confined and unconfined aquifer can entirely depleted)
    - The initialisation function initaquifercontus to set the component and describe the subsurface for all of CONTUS at a county-scale

# Subsurface model

*illustrations and description to come*

# Data

*sources description to come*